In [87]:
import pandas as pd
import numpy as np

In [88]:
name = 'graph_players_exp'
exps = ['4', '6', '5', '7', '7_temp', '8', '8_temp', '9', '10', '15', '16']
exp_dfs = []

for exp in exps:
    exp_dfs.append(pd.read_csv(f"{name}_{exp}.csv"))

df = pd.concat(exp_dfs)
df['game_setup'] = df['impostor_type'] + ', ' + df['innocent_type']

In [89]:
# leave 50 of each

In [90]:
assert (df['runtime'].nunique()) == (df['game_setup'].value_counts()/5).sum()

In [91]:
len_before_merge = len(df)

game_stats = df.groupby('runtime').apply(lambda group: pd.Series({
    'innocent_banished_cnt': (group['banished'] & ~group['killer']).astype(float).sum(),
    'killer_banished_flg': (group['killer'] & group['banished']).any().astype(float)
})).reset_index()

assert (df['runtime'].nunique()) == (df['game_setup'].value_counts()/5).sum()
assert (len(game_stats) == int((df['game_setup'].value_counts()/5).sum()))

df = df.merge(
    game_stats[['runtime', 'innocent_banished_cnt', 'killer_banished_flg']],
    on='runtime',
    how='inner'
).reset_index()

assert (len(df) == len_before_merge)

In [92]:
df['player_type'] = np.where(df['killer'], df['impostor_type'], df['innocent_type'])
# df['duplicate_search_rate'] = df['duplicate_search_rate'].fillna(0)
# df['vote_rate_for_killer'] = df['vote_rate_for_killer'].fillna(0)
# df['witness_vote_rate_for_killer'] = df['witness_vote_rate_for_killer'].fillna(0)

In [93]:
# import plotly.graph_objects as go
from IPython.display import display

def bootstrap_ci(data, n_iterations=1000, conf=0.9):
    means = []
    n_size = len(data)
    alpha = (1 - conf) / 2 * 100
    
    for _ in range(n_iterations):
        sample = np.random.choice(data, size=n_size, replace=True)
        means.append(np.mean(sample))
    
    mean = np.mean(means)
    lower = np.percentile(means, alpha)
    upper = np.percentile(means, 100 - alpha)

    return (lower, upper)

def crosstab_table(df: pd.DataFrame, target: str, access_impostor=True):
    player_types = ['Player', 'RelGraphPlayer', 'AriGraphPlayer']
    table_data = pd.DataFrame(index=player_types, columns=['Overall'] + player_types)
    exact_ci = {}

    if access_impostor:
        i_col = 'impostor_type'
        j_col = 'innocent_type'
    else:
        j_col = 'impostor_type'
        i_col = 'innocent_type'
    
    for i in player_types:
        subset = df[df[i_col] == i][target].dropna()
        ci = bootstrap_ci(subset) if not subset.empty else None
        table_data.loc[i, 'Overall'] = f"[{ci[0]:.2f}, {ci[1]:.2f}]" if ci else "None"
        
        exact_ci[i] = {}
        exact_ci[i]['Overall'] = ci
        for j in player_types:
            subset = df[(df[j_col] == j) & (df[i_col] == i)][target].dropna()
            ci = bootstrap_ci(subset) if not subset.empty else None
            table_data.loc[i, j] = f"[{ci[0]:.2f}, {ci[1]:.2f}]" if ci else "None"
            exact_ci[i][j] = ci

    print(table_data)

crosstab_table(df[df['killer']], 'num_killed')
crosstab_table(df[df['killer']], 'killer_banished_flg')

                     Overall        Player RelGraphPlayer AriGraphPlayer
Player          [1.95, 2.21]  [1.92, 2.34]   [1.94, 2.38]   [1.68, 2.18]
RelGraphPlayer  [1.82, 2.12]  [1.73, 2.13]   [2.34, 2.86]   [1.45, 1.89]
AriGraphPlayer  [1.99, 2.27]  [1.88, 2.32]   [2.00, 2.40]   [1.44, 2.33]
                     Overall        Player RelGraphPlayer AriGraphPlayer
Player          [0.21, 0.33]  [0.14, 0.34]   [0.20, 0.42]   [0.16, 0.36]
RelGraphPlayer  [0.21, 0.34]  [0.25, 0.48]   [0.03, 0.21]   [0.20, 0.39]
AriGraphPlayer  [0.21, 0.35]  [0.20, 0.42]   [0.20, 0.40]   [0.00, 0.33]


In [94]:
crosstab_table(df[~df['killer']], 'escaped', access_impostor=False)

                     Overall        Player RelGraphPlayer AriGraphPlayer
Player          [0.12, 0.17]  [0.12, 0.20]   [0.12, 0.21]   [0.08, 0.15]
RelGraphPlayer  [0.07, 0.11]  [0.04, 0.10]   [0.04, 0.12]   [0.07, 0.14]
AriGraphPlayer  [0.20, 0.26]  [0.14, 0.23]   [0.21, 0.31]   [0.14, 0.36]


In [95]:
crosstab_table(df[~df['killer']], 'duplicate_search_rate', access_impostor=False)

                     Overall        Player RelGraphPlayer AriGraphPlayer
Player          [0.08, 0.11]  [0.08, 0.14]   [0.06, 0.10]   [0.07, 0.12]
RelGraphPlayer  [0.15, 0.20]  [0.14, 0.22]   [0.12, 0.22]   [0.13, 0.22]
AriGraphPlayer  [0.04, 0.06]  [0.03, 0.07]   [0.04, 0.07]   [0.00, 0.06]


In [96]:
crosstab_table(df[~df['killer']], 'vote_rate_for_killer', access_impostor=False)

                     Overall        Player RelGraphPlayer AriGraphPlayer
Player          [0.45, 0.53]  [0.35, 0.47]   [0.51, 0.64]   [0.42, 0.54]
RelGraphPlayer  [0.45, 0.53]  [0.46, 0.58]   [0.31, 0.46]   [0.45, 0.58]
AriGraphPlayer  [0.45, 0.53]  [0.39, 0.52]   [0.48, 0.61]   [0.18, 0.50]


In [97]:
crosstab_table(df[~df['killer']], 'witness_vote_rate_for_killer', access_impostor=False)

                     Overall        Player RelGraphPlayer AriGraphPlayer
Player          [0.49, 0.64]  [0.16, 0.40]   [0.67, 0.89]   [0.47, 0.72]
RelGraphPlayer  [0.49, 0.68]  [0.43, 0.72]   [0.15, 0.65]   [0.50, 0.80]
AriGraphPlayer  [0.67, 0.87]  [0.62, 0.90]   [0.64, 0.93]   [0.33, 1.00]


In [98]:
crosstab_table(df[~df['killer']], 'innocent_banished_cnt', access_impostor=False)

                     Overall        Player RelGraphPlayer AriGraphPlayer
Player          [0.52, 0.58]  [0.56, 0.68]   [0.39, 0.50]   [0.55, 0.66]
RelGraphPlayer  [0.59, 0.66]  [0.56, 0.68]   [0.70, 0.82]   [0.50, 0.62]
AriGraphPlayer  [0.51, 0.58]  [0.53, 0.64]   [0.43, 0.54]   [0.67, 0.89]


In [99]:
df[df['innocent_type'] == df['impostor_type']][['innocent_type', 'runtime']].groupby('innocent_type').mean()

,runtime
innocent_type,
AriGraphPlayer,769.007697
Player,38.876689
RelGraphPlayer,865.180591
